In [4]:
import pandas as pd
import spacy, re
from spacy.attrs import ENT_IOB
from spacy.matcher import PhraseMatcher
from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.symbols import ORTH,LEMMA,POS
from pathlib import Path
from spacy.tokens import Span
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from spacy.strings import StringStore
import json
from collections import Counter
import pickle

### data preparation

In [ ]:
#import training data
recordDictBack = []
jsonFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\data\\training\\2020-12-04_LablledStr.json"

with open(jsonFile, 'r', encoding='utf-8') as input_file:
    for row in input_file.readlines():
        recordDictBack.append(json.loads(row))

#all desc
descList = [i['content'] for i in recordDictBack]

#all tag
tagList = [i['tagList'] for i in recordDictBack]

In [ ]:
#from yan ling - extra 200 rows
yanLingJsonFolder = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\data\\training\\Labelled_200_20201110"

yanLingRecord = []
for yanLingFile in os.listdir(yanLingJsonFolder):
    with open(yanLingJsonFolder + '\\' + yanLingFile) as input_file:
        for row in input_file.readlines():
            yanLingRecord.append(json.loads(row))    

yanLingFinalRecord = cleanRecord(yanLingRecord)

### munirah's processing pipeline and model

In [ ]:
def change_case_gen_pattern(parsed_doc):
    strg_2=''
    passport=False
    
    #   Change format of capitalized words into proper case (specifically nouns: NNP, NNS, NNPS) 
    for token in parsed_doc:
        tagged_sent = [(token.text, token.tag_)]
        
        normalized_sent = [w.capitalize() if (t in ["NNP","NNS","NNPS"] and w.isupper()) else w for (w,t) in tagged_sent]
        strg = re.sub(" (?=[\.,'!?:;])", "", ' '.join(normalized_sent))   

        lowerList = ('name','cash','bank','inter','amlatfa','cdd','jalan','ic','cust','business','director', 'sole', 'proprietor','str', 'cdm','saving', 'rm', 'myr','place','branch')

        if any(substring in strg.lower() for substring in lowerList):
               strg=strg.lower()
                
        # convert the remaining uppercase to lowercase
        if strg.isupper():
            strg = strg.lower()
        
        # remove unnecessary punctuations 
        strg_2 += re.sub(r'[=*()]',r'',strg.strip())
        strg_2 += ' '
        
            

    # remove space before certain symbols like <space><.> or <space><,>
    strg_2=re.sub(r'\s+([?.!,:"/\'])', r'\1', strg_2)
    strg_2=strg_2.replace("Õ"," ").replace("ð"," ").replace("õ"," ")
    strg_2=strg_2.replace("  ",". ")
     # remove space after certain symbols like <space><.> or <space><,>
    strg_2=re.sub(r'([?/])+\s', r'\1', strg_2)
    strg_2=strg_2.replace('"','\\"')
    
    if(strg_2.lower().__contains__('passport')):
        passport=True
    phrase_matcher(strg_2, passport)
    
#     print ('\n')
    
    return strg_2.strip()


# phrase matcher: shape
def phrase_matcher(strg, p_e):
    nlp = English()
    
    matcher = PhraseMatcher(nlp.vocab, attr="SHAPE")
    matcher.add(u"STR_ID", None, nlp(u"aa/025/s/2016/000019"), nlp(u"aa/025/s/2016/000075"))
    matcher.add(u"PERSON_ID", None, nlp(u"881102-08-5192"))
#     matcher.add(u"PERSON_ID_2", None, nlp(u"661124085949"), nlp(u"710905125067"))
    matcher.add(u"PASSPORT", None, nlp(u"r711493"), nlp(u"ma438972"), nlp(u"a3894268"))
    matcher.add(u"PASSPORT_OR_NRIC", None, nlp(u"706251339"), nlp(u"720201106027"), nlp(u"nric: 811018105265"), nlp(u"710905125067"))
    stringstore = StringStore([u"STR_ID",u"PERSON_ID",u"PASSPORT",u"PASSPORT_OR_NRIC"])
    
#     doc = nlp(strg.lower())
#     for match_id, start, end in matcher(doc):
#         span = doc[start:end]
#     #         with open("D:\\Users\\mcazwan\\Desktop\\fisitti\\str-id-patterns.jsonl", "a") as text_file:
#     #             text_file.write('{\n"label": "STR_ID", [{"TEXT":' + span.text + '}]),\n')
        
#         #  only allow 12 characters for person_id
#         if match_id == stringstore[u"PERSON_ID"] or match_id == stringstore[u"PERSON_ID_2"]:
#             if(len(span.text.replace(' ','').replace('-','')) == 12):
#                 print("(person_id):", doc[start:end])
#         elif match_id == stringstore[u"PASSPORT"] and p_e == True:
#         # check pattern
#             if(7 <= len(span.text.replace(' ','').replace('-','')) <= 9):
#                 print("(passport):", doc[start:end])
#         elif match_id == stringstore[u"PASSPORT_OR_NRIC"]:
#             if(7 <= len(span.text.replace(' ','').replace('-','')) <= 9) and p_e == True:
#                 print("(passport-digit):", doc[start:end])
                
#             elif(len(span.text.replace(' ','').replace('-','')) == 12):
#                 ic=span.text.replace(' ','').replace('-','')
#                 if(int(ic[2:4]) <= 12 and int(ic[4:6]) <= 31 and int(ic[6:8]) <= 14):
#                     print("(nric-digit):", doc[start:end])
#         elif match_id == stringstore[u"STR_ID"]:
#             print("(str_id):", doc[start:end])
             

# fix spaces
def fix_space_tags(doc):
    ent_iobs = doc.to_array([ENT_IOB])
    for i, token in enumerate(doc):
        if token.is_space:
            # Sets 'O' tag (0 is None, so I is 1, O is 2)
            ent_iobs[i] = 2
    
    doc.from_array([ENT_IOB], ent_iobs.reshape((len(doc), 1)))
    return doc

def prevent_sentence_boundaries(doc):
    for token in doc:
        if not can_be_sentence_start(token):
            token.is_sent_start = False
    return doc


def can_be_sentence_start(token):
#     print(token.text,':',token.i,'\n')
#     create separator
    
    if token.i == 0:
        return True
    # We're not checking for is_title here to ignore arbitrary titlecased
    # tokens within sentences
    
    elif token.nbor(-1).is_punct and token.nbor(-1).text not in [':','/','-']:
        return True

    elif token.nbor(-1).is_space:
        return True
    
    else:
        return False

spacyModel = 'en_core_web_md'
nlp = spacy.load(spacyModel, disable=['ner'])
nlp.add_pipe(prevent_sentence_boundaries, before="parser")

#     for other processing   
munirahModel = 'D:\\Users\\figohjs\\Documents\\NLP\\Spacy\\NER\\trained-model\\NER_All_Labels_lg'
nlp2 = spacy.load(munirahModel)
nlp2.add_pipe(fix_space_tags, name="fix-ner", before="ner")
nlp2.add_pipe(prevent_sentence_boundaries, before="parser")

jsonlFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\Notebook\\patterns.jsonl"
new_ruler = EntityRuler(nlp).from_disk(jsonlFile)
nlp2.add_pipe(new_ruler, after='ner')

In [ ]:
tagList[0]

In [ ]:
descList[0]

In [ ]:
#prediction
doc = nlp.pipe(iter(descList), batch_size=10, n_threads=4000) 

result = []
for parsed_doc in doc:
    #  add pipe
#     doc_2 = nlp2(change_case(parsed_doc))
    doc_2 = nlp2(change_case_gen_pattern(parsed_doc))
    result.append(doc_2)

predResultMunirah = []
for record in result:
    tempList = []
    for entity in record.ents:
        if entity.label_ in ['PERSON', 'ORG']:
            tempList.append((entity, entity.label_))
    predResultMunirah.append(tempList)

#convert result and tag into str for saving purpose
predResultMunirah2 = [[(str(j[0]),str(j[1])) for j in i] for i in predResultMunirah]

In [ ]:
#convert prediction, real result and desc into dict for file saving purpose
dfTag = pd.DataFrame()
dfTag['Description'] = descList
dfTag['RealTag'] = tagList
dfTag['PredTagMunirah'] = predResultMunirah2
recordDict = dfTag.to_dict(orient = 'record')

In [ ]:
len(recordDict)

In [ ]:
#writing list of dicts in a json file
jsonFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\data\\training\\2020-12-09_MunirahPrediction614.json"

with open(jsonFile, 'w', encoding='utf-8') as output_file:
    for dic in recordDict:
        json.dump(dic, output_file) 
        output_file.write("\n")

### CRF

### Trained with external data

In [1]:
from Script.crfProcessing import *
import json
import os
import pickle
import pandas as pd
from collections import Counter

In [2]:
#import training data
recordDictBack = []
jsonFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\data\\training\\2020-12-04_LablledStr.json"

with open(jsonFile, 'r', encoding='utf-8') as input_file:
    for row in input_file.readlines():
        recordDictBack.append(json.loads(row))

#all desc
descList = [i['content'] for i in recordDictBack]

#all tag
tagList = [i['tagList'] for i in recordDictBack]

In [3]:
#save model in pickle format
# filename = 'D:/Users/figohjs/Documents/NLP/NER/Model/2020-11-09_CRFmodel.sav'
filename = 'D:/Users/figohjs/Documents/NLP/NER/Model/2020-01-14_CRFmodel_Externalv1.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [4]:
predResultCRF = []
#loop every desc:
for no, desc in enumerate(descList):
    try:
        #processing
        cleanDesc = cleanText(desc)
        cleanDesc = changeCase(cleanDesc)
        cleanDesc = changeCase2(cleanDesc)
        cleanDesc = changeCase3(cleanDesc)
        #features engineering
        desc2 = [(i,) for i in cleanDesc.split(' ')]
        descFeatures = [sent2features(i) for i in [desc2]]
        #prediction
        y_pred = loaded_model.predict(descFeatures)
        #further processing
        predList = getNamedEntity(y_pred, [cleanDesc.split(' ')])[0]
        predResultCRF.append(predList)
    except Exception as e:
        print(e )
        print('\n')
        print('Description no:%s'%str(no))
        print('\n')
        print(desc)
        break

In [5]:
#convert prediction, real result and desc into dict for file saving purpose
dfTag = pd.DataFrame()
dfTag['Description'] = descList
dfTag['RealTag'] = tagList
dfTag['PredTagCRF'] = predResultCRF
recordDict = dfTag.to_dict(orient = 'record')

In [6]:
#writing list of dicts in a json file
# jsonFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\data\\training\\2020-12-17_CRFPrediction614.json"
jsonFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\data\\training\\2021-01-14_CRFPrediction614.json"

with open(jsonFile, 'w', encoding='utf-8') as output_file:
    for dic in recordDict:
        json.dump(dic, output_file) 
        output_file.write("\n")

### Trained with internal data

In [8]:
#default features used in nltk
def word2features(sent, i):
    word = str(sent[i][0])
#     postag = sent[i][1]

    features = {
        'bias': 1.0,
#         'ori':word,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
#         'postag': postag,
#         'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = str(sent[i-1][0])
#         postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
#             '-1:postag': postag1,
#             '-1:postag[:2]': postag1[:2],
        })
    else:
        #beginning of speech
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = str(sent[i+1][0])
#         postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
#             '+1:postag': postag1,
#             '+1:postag[:2]': postag1[:2],
        })
    else:
        #end of speech
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

def generateFeatures(descList):
    featuresList = []
    
    for desc in descList: 
        sample = [(i,) for i in desc.split(' ')]
        sampleFeatures = [sent2features(i) for i in [sample]]
        featuresList.append(sampleFeatures[0])
    
    return featuresList

# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, t) for w, t in zip(s['Token'].values.tolist(),
                                                s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence").apply(agg)
        self.sentences = [s for s in self.grouped]

In [6]:
import json
import os
import pickle
import pandas as pd
from collections import Counter
import re

In [10]:
filename = 'D:/Users/figohjs/Documents/NLP/NER/Model/2021-01-14_CRFmodel_Internalv1.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [3]:
#from yan ling - extra 200 rows
yanLingJsonFolder = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\data\\training\\Labelled_200_20201110"

yanLingRecord = []
for yanLingFile in os.listdir(yanLingJsonFolder):
    with open(yanLingJsonFolder + '\\' + yanLingFile) as input_file:
        for row in input_file.readlines():
            yanLingRecord.append(json.loads(row))    

#json file from NER label app contains duplicated record coz it s saved time by time
def cleanRecord(result): 
    finalList = []
    
    resultDict = [{i['content']:i['tagList'] for i in result}]
    dictKeys = [key for i in resultDict for key,val in i.items()]
    
    for finalKey in set(dictKeys):
        tempDict = {}
        #take first entry of records with same content
        tagVal = [val for i in resultDict for key, val in i.items() if key == finalKey][0]
        contentVal = finalKey
        tempDict['tagList'] = tagVal
        tempDict['content'] = contentVal
        finalList.append(tempDict)

    return finalList
            
yanLingFinalRecord = cleanRecord(yanLingRecord)

In [4]:
#all desc
descListYL = [i['content'] for i in yanLingFinalRecord]

#all tag
tagListYL = [i['tagList'] for i in yanLingFinalRecord]

In [12]:
#import stopword
stopWordList = []
txtFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\Data\\training\\stopwords.txt"
with open(txtFile, 'r') as myfile:
    for row in myfile.readlines():
        stopWordList.append(re.sub('\n','',row))

#import surname
surnameList = []
txtFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\Data\\training\\surname.txt"
with open(txtFile, 'r') as myfile:
    for row in myfile.readlines():
        surnameList.append(re.sub('\n','',row))

#tag dictionary
tagDict = {'org':'ORG', 'per':'PERSON', 'geo': 'GEO'}

def getNamedEntity(records, text):
    finalResult = []
    for noRow, row in enumerate(records):
        temp = []
        for noTerm, term in enumerate(row):
            #if token is beginning of org or per
            if term in ['B-' + i for i in tagDict.keys()]:
                tagType = term.split('-')[1]
                namedEnt = text[noRow][noTerm]
                #if current term is not the last term of the row
                if (noTerm + 1) != len(row):
                    if row[noTerm + 1] != ('I-' + tagType):
                        tempResult = checkTuple((namedEnt, tagDict[tagType]))
                        if tempResult:
                            temp.append(tempResult)
                            
                else:
                    tempResult = checkTuple((namedEnt, tagDict[tagType]))
                    if tempResult:
                        temp.append(tempResult)

            #if token is inside org or per
            elif term in ['I-org', 'I-per', 'I-geo']:
                tagType = term.split('-')[1]
                namedEnt = ' '.join([namedEnt, text[noRow][noTerm]])
                #if current term is not the last term of the row
                if (noTerm + 1) != len(row):
                    if row[noTerm + 1] != ('I-' + tagType):
                        tempResult = checkTuple((namedEnt, tagDict[tagType]))
                        if tempResult:
                            temp.append(tempResult)   

                else:
                    tempResult = checkTuple((namedEnt, tagDict[tagType]))
                    if tempResult:
                        temp.append(tempResult)
                        
        finalResult.append(temp)
            
    return finalResult  

def checkTuple(tupleResult):
    if tupleResult[1] in ['PERSON', 'GEO']:
        if re.search('berhad|bhd', tupleResult[0], flags = re.I):
            return (tupleResult[0], 'ORG')
        else:
            #filter out geo
            if tupleResult[1] == 'GEO':
                return None
            else:
                return tupleResult
            
    elif tupleResult[1] == 'ORG':
        #put chinese name back as label
        if len(tupleResult[0].split(' ')) == 3 and not re.search('berhad|bhd', tupleResult[0], flags = re.I):
            if tupleResult[0].split(' ')[0].lower() in surnameList:
                return (tupleResult[0], 'PERSON')
            else:
                return tupleResult
        else:
            return tupleResult
    else:
        return tupleResult

#first layer of text cleaning
def cleanText(text):
    #special chars list
    scList = ['\si.e.\s']
    
    #replace \n with ' '
    text = re.sub('\n', ' ', text)
    #remove null in end of sentence
    text = re.sub('null $', '', text, flags = re.I)
    #remove rm as training data does not have rm
    text = re.sub('(rm|myr)\s*(\d+)', r'\2', text, flags = re.I)
    #remove special char 
    text = re.sub('|'.join(scList), '', text)
    #remove additional spaces
    text = re.sub('(\s)+', r'\1', text)        
    return text

predResultCRF = []
#loop every desc:
for no, desc in enumerate(descListYL):
    try:
        cleanDesc = cleanText(desc)
        desc2 = [(i,) for i in cleanDesc.split(' ')]
        descFeatures = [sent2features(i) for i in [desc2]]
        y_pred = loaded_model.predict(descFeatures)
        #further processing
        predList = getNamedEntity(y_pred, [cleanDesc.split(' ')])[0]
        predResultCRF.append(predList)
        
    except Exception as e:
        print(e )
        print('\n')
        print('Description no:%s'%str(no))
        print('\n')
        print(desc)
        break

In [17]:
len(predResultCRF)

199

In [16]:
#convert prediction, real result and desc into dict for file saving purpose
dfTag = pd.DataFrame()
dfTag['Description'] = descList
dfTag['RealTag'] = tagList
dfTag['PredTagCRF'] = predResultCRF
recordDict = dfTag.to_dict(orient = 'record')

NameError: name 'descList' is not defined

### Evaluation

In [19]:
#load json data from munirah prediction file
jsonFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\data\\training\\2020-12-09_MunirahPrediction614.json"

munirahList = []

with open(jsonFile) as input_file:
    for row in input_file.readlines():
        munirahList.append(json.loads(row))   

#extract munirah prediction
predMunirahList = [i['PredTagMunirah'] for i in munirahList]

In [7]:
#load json data from munirah prediction file
# jsonFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\data\\training\\2020-12-09_CRFPrediction614.json"
# jsonFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\data\\training\\2020-12-17_CRFPrediction614.json"
jsonFile = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\data\\training\\2021-01-14_CRFPrediction614.json"

crfList = []

with open(jsonFile) as input_file:
    for row in input_file.readlines():
        crfList.append(json.loads(row))   

#extract munirah prediction
predCRFList = [i['PredTagCRF'] for i in crfList]

In [8]:
#convert bank to org in real tag
def changeBanktoOrg(result):
    realTagList = []
    for record in result:
        tempList = []
        for row in record['RealTag']:
            if row[1] == 'BANK':
                tempList.append([row[0], 'ORG', row[2]])
            else:
                tempList.append(row)
        realTagList.append(tempList)
    return realTagList

def calculateResult(pred, real, categoryList):
    finalResultList = []
    for no, record in enumerate(pred):
        resultDict = {}
        for category in categoryList:
            predList = [i for i in record if i[1] == category]
            realList = [i for i in real[no] if i[1] == category]
            #form dict for pred and real - groupby and count
            #ex:{'Affin Hwang-PERSON': 2, 'Lay Hong Berhad-ORG': 2} for 1 record
            predDict = dict(Counter([str(i[0]).lower() + '-' + i[1] for i in predList]))
            realDict = dict(Counter([str(i[0]).lower() + '-' + i[1] for i in realList]))
            numCorrect = sum([predDict[key] if predDict[key] <= realDict[key] else realDict[key]  
                              for key in predDict.keys() if key in realDict.keys()])
            totalEntity = sum([j for i,j in realDict.items()])
            totalPred = sum([j for i,j in predDict.items()])
            #total number of correct prediction, prediction and entities in real result by each category for each record
            resultDict[category] = (numCorrect, totalPred, totalEntity)
        
        #correct named entities but wrong label
        predList = [i[0] for i in record]
        realList = [i[0] for i in real[no]]
        predDict = dict(Counter([str(i[0]).lower() for i in predList]))
        realDict = dict(Counter([str(i[0]).lower() for i in realList]))        
        numCorrect = sum([predDict[key] if predDict[key] <= realDict[key] else realDict[key]  
                          for key in predDict.keys() if key in realDict.keys()])
        totalEntity = sum([j for i,j in realDict.items()])
        totalPred = sum([j for i,j in predDict.items()])
        #store result: number of correct named entities but wrong label
        resultDict['NE'] = (numCorrect, totalPred, totalEntity)
        
        #store result for each record
        finalResultList.append(resultDict)
    return finalResultList

def calculateEvalMetric(inputList, categoryList):
    #total number of correct prediction, prediction and entities in real result by each category for each record
    #input
    #initiate a dict
    resultDict = {}
    metricList = ['fn', 'fp', 'tp', 'Precision', 'Recall', 'F1Score']
    for category in categoryList:
        resultDict[category] = {}
        for metric in metricList:
            resultDict[category][metric] = 0
    
    #get tp, fp and fn for each category
    for record in inputList:
        for category in categoryList:
            resultDict[category]['tp']+=record[category][0]
            resultDict[category]['fp']+=record[category][1]-record[category][0]
            #if total of entities >= total prediction
            if record[category][2] >= record[category][1]:
                resultDict[category]['fn']+=record[category][2]-record[category][1]
            else:
                resultDict[category]['fn']+=0
                
    #get tp, fp and fn for all
    totalFN = sum([j['fn'] for i,j in resultDict.items()])
    totalFP = sum([j['fp'] for i,j in resultDict.items()])
    totalTP = sum([j['tp'] for i,j in resultDict.items()]) 
    
    allCategory = 'Overall'
    resultDict[allCategory] = {}
    resultDict[allCategory]['fn'] = totalFN 
    resultDict[allCategory]['fp'] = totalFP
    resultDict[allCategory]['tp'] = totalTP 
          
    #calculate precision, recall and f1score for each category and all
    for category in categoryList + [allCategory]:
        resultDict[category]['Precision'] = resultDict[category]['tp']/(resultDict[category]['tp'] + resultDict[category]['fp'])
        resultDict[category]['Recall'] = resultDict[category]['tp']/(resultDict[category]['tp'] + resultDict[category]['fn'])
        resultDict[category]['F1Score'] = 2*(resultDict[category]['Precision']*resultDict[category]['Recall'])\
                                            /(resultDict[category]['Precision'] + resultDict[category]['Recall'])
        
    return resultDict

In [22]:
#convert bank to org in real tag
munirahRealTagList = changeBanktoOrg(munirahList)

#total number of correct prediction, prediction and entities in real result by each category
finalMunirahList = calculateResult(pred = predMunirahList, real = munirahRealTagList, 
                                    categoryList = ['PERSON', 'ORG'])

metricMunirahDict = calculateEvalMetric(finalMunirahList, categoryList = ['PERSON', 'ORG', 'NE'])

In [9]:
#convert bank to org in real tag
crfRealTagList = changeBanktoOrg(crfList)

#total number of correct prediction, prediction and entities in real result by each category
finalCRFList = calculateResult(pred = predCRFList, real = crfRealTagList, 
                                    categoryList = ['PERSON', 'ORG'])

# finalCRFList

In [28]:
#convert bank to org in real tag
crfRealTagList = changeBanktoOrg(crfList)

#total number of correct prediction, prediction and entities in real result by each category
finalCRFList = calculateResult(pred = predCRFList, real = crfRealTagList, 
                                    categoryList = ['PERSON', 'ORG'])

metricCRFDict = calculateEvalMetric(finalCRFList, categoryList = ['PERSON', 'ORG', 'NE'])

dfCRF = pd.DataFrame(metricCRFDict)
# dfCRF.columns = [i + '_CRF' for i in dfCRF.columns]
dfCRF.reset_index(inplace = True)
dfCRF.rename(columns = {'index':'Metric'}, inplace = True)
filename = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\Data\\result\\2020-12-17_CRFMetrics.csv"
dfCRF.to_csv(filename, index = False)
dfCRF

,Metric,PERSON,ORG,NE,Overall
0,fn,857.000000,1130.000000,4219.000000,6206.000000
1,fp,1914.000000,2569.000000,2854.000000,7337.000000
2,tp,309.000000,185.000000,2123.000000,2617.000000
3,Precision,0.139001,0.067175,0.426562,0.262909
4,Recall,0.265009,0.140684,0.334752,0.296611
5,F1Score,0.182355,0.090931,0.375121,0.278745


In [11]:
#convert bank to org in real tag
crfRealTagList = changeBanktoOrg(crfList)

#total number of correct prediction, prediction and entities in real result by each category
finalCRFList = calculateResult(pred = predCRFList, real = crfRealTagList, 
                                    categoryList = ['PERSON', 'ORG'])

metricCRFDict = calculateEvalMetric(finalCRFList, categoryList = ['PERSON', 'ORG', 'NE'])

dfCRF = pd.DataFrame(metricCRFDict)
# dfCRF.columns = [i + '_CRF' for i in dfCRF.columns]
dfCRF.reset_index(inplace = True)
dfCRF.rename(columns = {'index':'Metric'}, inplace = True)
dfCRF

,Metric,PERSON,ORG,NE,Overall
0,fn,1013.000000,1076.000000,4336.000000,6425.000000
1,fp,1527.000000,2769.000000,2664.000000,6960.000000
2,tp,274.000000,206.000000,2112.000000,2592.000000
3,Precision,0.152138,0.069244,0.442211,0.271357
4,Recall,0.212898,0.160686,0.327543,0.287457
5,F1Score,0.177461,0.096782,0.376336,0.279175


In [29]:
dfSpacy = pd.DataFrame(metricMunirahDict)
# dfSpacy.columns = [i + '_SpaCy' for i in dfSpacy.columns]
dfSpacy.reset_index(inplace = True)
dfSpacy.rename(columns = {'index':'Metric'}, inplace = True)
filename = "D:\\Users\\figohjs\\Documents\\NLP\\NER\\Data\\result\\2020-12-17_MunirahMetrics.csv"
dfSpacy.to_csv(filename, index = False)
dfSpacy

,Metric,PERSON,ORG,NE,Overall
0,fn,459.000000,1471.000000,4851.000000,6781.000000
1,fp,1912.000000,739.000000,1986.000000,4637.000000
2,tp,1089.000000,552.000000,2306.000000,3947.000000
3,Precision,0.362879,0.427576,0.537279,0.459809
4,Recall,0.703488,0.272862,0.322202,0.367916
5,F1Score,0.478787,0.333132,0.402830,0.408761
